In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
data_dir = '/content/drive/MyDrive/ColabNotebooks/데이터분석캡스톤디자인' #경로 지정하기
file_list = os.listdir(data_dir)
print(file_list)

['유사도모델.ipynb', 'data.csv', 'ppomppu.csv', 'fmkorea_hotdeal.csv', '2english.csv', 'data_digital.csv', 'Make_data.ipynb', '2korea.csv', 'ppomppu_DigitClassfiy_fold0.pt', 'ppomppu_DigitClassfiy_fold1.pt', 'ppomppu_DigitClassfiy_fold2.pt', 'ppomppu_DigitClassfiy_fold3.pt', 'ppomppu_DigitClassfiy_fold4.pt', 'DistilBert.ipynb', 'PreProcessing+Model.ipynb']


In [ ]:
df =pd.read_csv(data_dir + '/ppomppu.csv')
df2 =pd.read_csv(data_dir + '/fmkorea_hotdeal.csv')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import re
s1 = re.compile('1+1')
s2 = re.compile('역대가')
p = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃…]')

def remove_special_characters(sentence, lower=True):
    sentence = s1.sub(' ', str(sentence))
    sentence = p.sub(' ', sentence) 
    sentence = s1.sub(' ', sentence)
    sentence = ' '.join(sentence.split())
    if lower:
        sentence = sentence.lower()
    return sentence

df['title'] = df['title'].map(remove_special_characters)
df2['title'] = df2['title'].map(remove_special_characters)

In [ ]:
df_full = pd.concat([df,df2],ignore_index=True)

In [ ]:
df_full.head()

,Unnamed: 0,Unnamed: 0.1,id,title,category,product_name,price,shopping_fee,hotdeal_place,link,img_url,shop_url,text,date,label
0,0,0.0,2_400748,아토세이프 섬유유연제 2 5l x 4개,[기타],아토세이프 섬유유연제 2.5L x 4개,"6,900원",무료배송,옥션,https://www.ppomppu.co.kr/zboard/view.php?id=p...,https://cdn.ppomppu.co.kr/images/personal_sett...,https://s.ppomppu.co.kr?idno=ppomppu_400748&en...,\n\n \n아토세이프 섬유유연제 2.5L x 4개 무료배송\n \n네이버경유하면...,21/10/21,0.0
1,1,1.0,2_400747,오바다 국내산 반건조오징어 10미,[식품/건강],오바다 국내산 반건조오징어 10미,"1.2kg내외)(19,740",무료,인터파크,https://www.ppomppu.co.kr/zboard/view.php?id=p...,no_image,https://s.ppomppu.co.kr?idno=ppomppu_400747&en...,쿠폰은 링크에서 바로 다운가능합니다\n수산대전 30%+10% 중복 총40%할인이네요\n,21/10/21,0.0
2,2,2.0,2_400746,휴지공방무형광 3겹 롤휴지 2개,[기타],휴지공방무형광 3겹 롤휴지 * 2개,"15,900",무료,11번가,https://www.ppomppu.co.kr/zboard/view.php?id=p...,https://cdn.ppomppu.co.kr/images/personal_sett...,https://s.ppomppu.co.kr?idno=ppomppu_400746&en...,\n요거 원래 한묶음에 9900원정도 하고\n평상시엔 두묶음에 2만원 이거든요 ~...,21/10/21,0.0
3,3,3.0,2_400745,파우트 노필터 초음파 미니가습기 타임딜,[가전/가구],파우트 노필터 초음파 미니가습기 타임딜,19900원,무배,터파크,https://www.ppomppu.co.kr/zboard/view.php?id=p...,https://cdn.ppomppu.co.kr/images/personal_sett...,https://s.ppomppu.co.kr?idno=ppomppu_400745&en...,요즘 건조해서 미니 가습기 하나 장만했는데 \n가격은 찾아본것중엔 젤 싼거 같아요 ...,21/10/21,0.0
4,4,4.0,2_400743,앤드지 세미 오버핏 트렌치 코트 1 1,[의류/잡화],앤드지 세미 오버핏 트렌치 코트 1+1,79710,무료,11번가,https://www.ppomppu.co.kr/zboard/view.php?id=p...,https://cdn.ppomppu.co.kr/images/personal_sett...,https://s.ppomppu.co.kr?idno=ppomppu_400743&en...,\n\n\n\n \n \n\n\n\n \n \n\n\n\n \n \n\n\n\n ...,21/10/21,0.0


In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_full['title'])

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df_full.index, index=df_full['title']).drop_duplicates()

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    product_indices = [i[0] for i in sim_scores]

    return df_full.iloc[product_indices]

In [ ]:
df_full.loc[2,"title"]

'휴지공방무형광 3겹 롤휴지 2개'

In [ ]:
import re
sameProduct = []
for j in range(10):
  products = get_recommendations(df_full.loc[j,"title"])
  products.reset_index(inplace=True)
  for i in range(len(products)):
    try :
      if re.sub(r'[^0-9]', '', products.loc[i,"price"]) == "":
        price1 = 0
      else:
        price1 = int(re.sub(r'[^0-9]', '', products.loc[i,"price"]))
      if re.sub(r'[^0-9]', '', df_full.loc[j,"price"]) == "":
        price2 = 0
      else:
        price2 = int(re.sub(r'[^0-9]', '', df_full.loc[j,"price"]))
    except:
      continue
    if products.loc[i,"hotdeal_place"] == df_full.loc[j,"hotdeal_place"] and abs(((price1 - price2) // price2))< 0.05:
      sameProduct += [[products.loc[i,"title"],df_full.loc[j,"title"]]]

In [ ]:
sameProduct

[['아토세이프 섬유유연제 13l', '아토세이프 섬유유연제 2 5l x 4개'],
 ['스마일 섬유유연제 후리지아향 2 5l 4개', '아토세이프 섬유유연제 2 5l x 4개'],
 ['크리넥스 데코앤소프트 3겹 27m 30롤 2팩', '휴지공방무형광 3겹 롤휴지 2개'],
 ['테이퍼드핏 스트링 팬츠', '지오다노 5 tr 테이퍼드핏 히든밴딩 슬랙스'],
 ['디스커버리 아동 구스다운', '에디션앤드지 구스다운 패딩코트']]